In [ ]:
import gc   #garbage collector-to free memory
gc.enable() #Activate
import zipfile #to unzip files
import pandas as pd

In [ ]:
!pip install --upgrade kaggle
from google.colab import files
from datetime import datetime
api_token=files.upload()

     |████████████████████████████████| 61kB 1.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=ce454d7e862ae1b9f4b00565f9443fbf5e1c846eace2bb588424cc5b705f6928
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


Saving kaggle.json to kaggle.json


In [ ]:
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c instacart-market-basket-analysis

 99% 195M/196M [00:06<00:00, 28.5MB/s]
100% 196M/196M [00:06<00:00, 31.4MB/s]


In [ ]:
with zipfile.ZipFile('/content/instacart-market-basket-analysis.zip','r')as z:
  z.extractall('.')

<h3><b>Features Extraction</b></H3>



1. Creating a dataframe for feature extraction.



1.1) I'm extracting features using only orders that were previously placed by customers [not in train and test orders] ,i.e. using only products_prior csv file 

In [ ]:
with zipfile.ZipFile('/content/order_products__prior.csv.zip','r')as z:
    z.extractall('.')

In [ ]:
order_products_prior=pd.read_csv('./order_products__prior.csv')
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


1.2) In order to get more information about these prior orders(like user_id,order_dow etc).I'm merging above dataframe with orders(after extracting it from orders.csv file)

In [ ]:
with zipfile.ZipFile('/content/orders.csv.zip','r')as z:
    z.extractall('.')

In [ ]:
orders=pd.read_csv('./orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0




*   Final dataframe for extraction  



In [ ]:
combine_df=orders.merge(prior_orders,on='order_id',how='inner')
combine_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [ ]:
combine_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
 #   Column                  Non-Null Count     Dtype  
---  ------                  --------------     -----  
 0   order_id                32434489 non-null  int64  
 1   user_id                 32434489 non-null  int64  
 2   eval_set                32434489 non-null  object 
 3   order_number            32434489 non-null  int64  
 4   order_dow               32434489 non-null  int64  
 5   order_hour_of_day       32434489 non-null  int64  
 6   days_since_prior_order  30356421 non-null  float64
 7   product_id              32434489 non-null  int64  
 8   add_to_cart_order       32434489 non-null  int64  
 9   reordered               32434489 non-null  int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 2.7+ GB


1.3) replacing nan values with 0.0 as it indicates this is first order placed by that order or days since last order =0.

In [ ]:
import numpy as np
combine_df['days_since_prior_order'].replace(to_replace=np.nan,value=0,inplace=True)
combine_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,0.0,196,1,0
1,2539329,1,prior,1,2,8,0.0,14084,2,0
2,2539329,1,prior,1,2,8,0.0,12427,3,0
3,2539329,1,prior,1,2,8,0.0,26088,4,0
4,2539329,1,prior,1,2,8,0.0,26405,5,0


1.4) deleting prior_orders and orders datafame as no longer required.

In [ ]:
del prior_orders

In [ ]:
del orders


2.   USER 'S Features Extraction



**2.1)Number of orders placed per user**

2.1.1) in order to get this feature,i'm grouping combine_df on user_id and selecting max value of order_number col for each user.

In [ ]:
user_num_orders=combine_df.groupby('user_id')['order_number'].max().reset_index(name='Total orders')  #df- user_id, number of  orders per user

In [ ]:
user_num_orders.head()

,user_id,Total orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


**2.2) User's Reorder Ratio**

 user reorder ratio =
                          Total number of times that user reorder products
                          -----------------------------------------------
                          Total number of purchased products by that user

In [ ]:
#total number of times user reorder products :given by sum of reordered col(as it contains 0-for not reorder and 1= reorder) per user
user_num_reorder=combine_df.groupby('user_id')['reordered'].sum().reset_index(name='user_reorders')

In [ ]:
user_num_reorder.head()

,user_id,user_reorders
0,1,41
1,2,93
2,3,55
3,4,1
4,5,14


In [ ]:
#total number of products purchased by user :given by counting number of values in product_id per user
user_total_purchases=combine_df.groupby('user_id')['product_id'].count().reset_index(name='user_total_purchases')

In [ ]:
user_total_purchases.head()

,user_id,user_total_purchases
0,1,59
1,2,195
2,3,88
3,4,18
4,5,37


In [ ]:
#required ratio
user_num_reorder['user_re_ratio']=user_num_reorder['user_reorders'].div(user_total_purchases['user_total_purchases'],axis=0)

In [ ]:
user_num_reorder.head()

,user_id,user_reorders,user_re_ratio
0,1,41,0.694915
1,2,93,0.476923
2,3,55,0.625000
3,4,1,0.055556
4,5,14,0.378378


**2.3) User's Non Reorder Ratio**

 user non-reorder ratio =
                          Total number of times that user not reorder products
                          -----------------------------------------------
                          Total number of products purchased by that user

In [ ]:
#total number of times user not reorder products :given by number of values with  reordered== 0 per user
user_num_non_reorder=combine_df[combine_df.reordered==0].groupby('user_id')['reordered'].count().reset_index(name='user_non_reorders')

In [ ]:
user_num_non_reorder.head()

,user_id,user_non_reorders
0,1,18
1,2,102
2,3,33
3,4,17
4,5,23


In [ ]:
#required ratio
user_num_non_reorder['user_ntre_ratio']=user_num_non_reorder['user_non_reorders'].div(user_total_purchases['user_total_purchases'],axis=0)

In [ ]:
user_num_non_reorder.head()

,user_id,user_non_reorders,user_ntre_ratio
0,1,18,0.305085
1,2,102,0.523077
2,3,33,0.375000
3,4,17,0.944444
4,5,23,0.621622




> Combining above 3 users' features into a dataframe



In [ ]:
user_df=user_num_orders.merge(user_num_reorder,on='user_id',how='left').merge(user_num_non_reorder,on='user_id',how='left')
user_df.head()

,user_id,Total orders,user_reorders,user_re_ratio,user_non_reorders,user_ntre_ratio
0,1,10,41,0.694915,18,0.305085
1,2,14,93,0.476923,102,0.523077
2,3,12,55,0.625000,33,0.375000
3,4,5,1,0.055556,17,0.944444
4,5,4,14,0.378378,23,0.621622


In [ ]:
user_df.drop(['user_reorders','user_non_reorders'],axis=1,inplace=True)  #droping extra cols 
user_df.head()  #final dataframe

,user_id,Total orders,user_re_ratio,user_ntre_ratio
0,1,10,0.694915,0.305085
1,2,14,0.476923,0.523077
2,3,12,0.625000,0.375000
3,4,5,0.055556,0.944444
4,5,4,0.378378,0.621622


In [ ]:
user_df.shape

(206209, 4)

In [ ]:
user_df.to_csv('/content/drive/MyDrive/Instacart/user.csv')



> Deleting drames from 2.1 ,2.2, 2.3 as no longer needed



In [ ]:
del user_num_orders
gc.collect()

938

In [ ]:
del user_num_non_reorder
gc.collect()

295

In [ ]:
del user_num_reorder
gc.collect()

486

In [ ]:
del user_total_purchases
gc.collect()

527


 

3.   User-Product's Features





**3.1)Product reorder ratio per user**

prd_re_ratio_user=
                   Total times the product purchased by a user
                   ------------------------------------------------
                   Total times this user placed any order since the product's first purchase order



*   for numerator:




In [ ]:
#total times user order this prd - given by count of order id col per  user per prd
user_prd_reorders=combine_df.groupby(['user_id','product_id'])[['order_id']].count().reset_index()

In [ ]:
user_prd_reorders.head()

,user_id,product_id,order_id
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3




*   for denominator



In [ ]:
#step 1-total orders for this user
user_num_orders=combine_df.groupby('user_id')['order_number'].max().reset_index(name='Total orders')  #df- user_id, number of  orders per user

In [ ]:
user_num_orders.head()

,user_id,Total orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [ ]:
#step 2- first time(on which order) user order this product- given by first order number per user per product
user_prd_first=combine_df.groupby(['user_id','product_id'])[['order_number']].min().reset_index()

In [ ]:
user_prd_first.head()  #here order_number is order number of first purchase of this prd

,user_id,product_id,order_number
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [ ]:
#merging user total orders - using right join
import pandas as pd
user_prd_first=pd.merge(user_prd_first,user_num_orders,on='user_id',how='right')
user_prd_first.head()

,user_id,product_id,order_number,Total orders
0,1,196,1,10
1,1,10258,2,10
2,1,10326,5,10
3,1,12427,1,10
4,1,13032,2,10


In [ ]:
#step 3-total products purchased by user after this product purchase==total_purchases-order_number+1
#as order_number == first time which order user purchased this product

In [ ]:
user_prd_first['after_prd_orders']=user_prd_first['Total orders']-user_prd_first['order_number']+1

In [ ]:
user_prd_first.head()

,user_id,product_id,order_number,Total orders,after_prd_orders
0,1,196,1,10,10
1,1,10258,2,10,9
2,1,10326,5,10,6
3,1,12427,1,10,10
4,1,13032,2,10,9




*   Final dataframe



In [ ]:
user_prd_reorders['user_prd_re_ratio']=user_prd_reorders['order_id'].div(user_prd_first['after_prd_orders'],axis=0)

In [ ]:
user_prd_reorders.head()

,user_id,product_id,order_id,user_prd_re_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


In [ ]:
#final dataframe=u_p_df
u_p_df=user_prd_reorders.drop('order_id',axis=1)  #droping not required col
u_p_df.head()

,user_id,product_id,user_prd_re_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [ ]:
del user_prd_first
del user_prd_reorders

**3.2) Times a user added a product to cart at position 1 ,2 or 3**

                  =Number of times a user added this product to cart position 1 , 2 and 3 in all his orders.

In [ ]:
#step 1-filtering rows with cart position 1 ,2 and 3 only
cart=combine_df[combine_df['add_to_cart_order']<=3]
cart.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,0.0,196,1,0
1,2539329,1,prior,1,2,8,0.0,14084,2,0
2,2539329,1,prior,1,2,8,0.0,12427,3,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0


In [ ]:
#step2- selecting count of add_to_cart_order per user per product 
times=cart.groupby(['user_id','product_id'])['add_to_cart_order'].count().reset_index()
times.head()

,user_id,product_id,add_to_cart_order
0,1,196,9
1,1,10258,7
2,1,12427,8
3,1,14084,1
4,1,25133,1


In [ ]:
del cart



*   Final dataframe after step 3.1 and 3.2



In [ ]:
u_p_df1=pd.merge(u_p_df,times,on=['user_id','product_id'],how='left')
u_p_df1.head()

,user_id,product_id,user_prd_re_ratio,add_to_cart_order
0,1,196,1.000000,9.0
1,1,10258,1.000000,7.0
2,1,10326,0.166667,NaN
3,1,12427,1.000000,8.0
4,1,13032,0.333333,NaN


In [ ]:
del times
del u_p_df

In [ ]:
#replacing null values with 0 => as it indicates this prd hasn't ordered at 1, 2, 3 position at all
u_p_df1['add_to_cart_order'].replace(to_replace=np.nan,value=0,inplace=True)
#changing col name from add_to_cart_order to times_cart_123
u_p_df1.rename({'add_to_cart_order':'times_cart_123'},axis=1,inplace=True)
u_p_df1.head()

,user_id,product_id,user_prd_re_ratio,times_cart_123
0,1,196,1.000000,9.0
1,1,10258,1.000000,7.0
2,1,10326,0.166667,0.0
3,1,12427,1.000000,8.0
4,1,13032,0.333333,0.0


**3.3)Times user bought the product in last 5 orders**

          =Number of times a user bought this product in his last 5 orders.



```
# Step -1 to select last 5 orders of a user
```



In [ ]:
#create a col 'reverse_order_number' that indicates order numbers in reverse for a user.
#this is achieved by: (max orders of the user) - (current order number) +1
combine_df['reverse_order_number']=combine_df.groupby('user_id')['order_number'].transform(max) - combine_df.order_number +1
#.transform(max) gives highest of col order_number for each user
#and we subtract order number of each row.
combine_df.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,reverse_order_number
0,2539329,1,prior,1,2,8,0.0,196,1,0,10
1,2539329,1,prior,1,2,8,0.0,14084,2,0,10
2,2539329,1,prior,1,2,8,0.0,12427,3,0,10
3,2539329,1,prior,1,2,8,0.0,26088,4,0,10
4,2539329,1,prior,1,2,8,0.0,26405,5,0,10
5,2398795,1,prior,2,3,7,15.0,196,1,1,9
6,2398795,1,prior,2,3,7,15.0,10258,2,0,9
7,2398795,1,prior,2,3,7,15.0,12427,3,1,9
8,2398795,1,prior,2,3,7,15.0,13176,4,0,9
9,2398795,1,prior,2,3,7,15.0,26088,5,1,9


In [ ]:
#now select only orders with reverse_order_number = 1-5
rev_df=combine_df[combine_df['reverse_order_number']<=5]
rev_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,reverse_order_number
29,3367565,1,prior,6,2,7,19.0,196,1,1,5
30,3367565,1,prior,6,2,7,19.0,12427,2,1,5
31,3367565,1,prior,6,2,7,19.0,10258,3,1,5
32,3367565,1,prior,6,2,7,19.0,25133,4,1,5
33,550135,1,prior,7,1,9,20.0,196,1,1,4




```
# Step 2 - Selecting times a user ordered a product in his last 5 orders
```



In [ ]:
#counting numbers of orders_id  per user per product in his last 5 orders
prd_user_last_5=rev_df.groupby(['user_id','product_id'])['order_id'].count().reset_index()
prd_user_last_5.head()

,user_id,product_id,order_id
0,1,196,5
1,1,10258,5
2,1,12427,5
3,1,13032,2
4,1,25133,5


In [ ]:
del rev_df



*   Merging this df with final df of step 3.2



In [ ]:
u_p_df2=pd.merge(u_p_df1,prd_user_last_5,on=['user_id','product_id'],how='left')
u_p_df2.head()

,user_id,product_id,user_prd_re_ratio,times_cart_123,order_id
0,1,196,1.000000,9.0,5.0
1,1,10258,1.000000,7.0,5.0
2,1,10326,0.166667,0.0,NaN
3,1,12427,1.000000,8.0,5.0
4,1,13032,0.333333,0.0,2.0


In [ ]:
del prd_user_last_5

In [ ]:
del u_p_df1

In [ ]:
#replacing null values with 0 => as it indicates this prd hasn't ordered in last 5 orders
u_p_df2['order_id'].replace(to_replace=np.nan,value=0,inplace=True)
#changing col name from order_id to times_prd_last_5
u_p_df2.rename({'order_id':'times_prd_last_5'},axis=1,inplace=True)

In [ ]:
u_p_df2.head()

,user_id,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5
0,1,196,1.000000,9.0,5.0
1,1,10258,1.000000,7.0,5.0
2,1,10326,0.166667,0.0,0.0
3,1,12427,1.000000,8.0,5.0
4,1,13032,0.333333,0.0,2.0


In [ ]:
u_p_df2.shape

(13307953, 5)

In [ ]:
u_p_df2.to_csv('/content/drive/MyDrive/Instacart/uxp.csv')



4.Day_of_week's  Features   



**4.1)Day of week reorder ratio per user**

                   =Total times user reordered at this day
                   -----------------------------------------------
                   Total times user placed order on this day

In [ ]:
user_day=combine_df.groupby(['user_id','order_dow'])['reordered'].mean().reset_index()

In [ ]:
user_day.head()

,user_id,order_dow,reordered
0,1,1,0.882353
1,1,2,0.444444
2,1,3,0.545455
3,1,4,0.727273
4,2,1,0.671875


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**4.2) Day of week not reorder ratio**

                       =Total times not reorders on a day
                       ----------------------------------
                       Total orders on a day

In [ ]:
#total orders per day
day_total=combine_df.groupby('order_dow')['order_number'].count().reset_index()

In [ ]:
day_total.head()

,order_dow,order_number
0,0,6209666
1,1,5665856
2,2,4217798
3,3,3844117
4,4,3787215


In [ ]:
#total not reorders per day
day_nt=combine_df[combine_df.reordered==0].groupby('order_dow')['order_number'].count().reset_index()

In [ ]:
day_nt.head()

,order_dow,order_number
0,0,2575300
1,1,2244571
2,2,1730261
3,3,1590419
4,4,1549049


In [ ]:
day_nt['ntre_ratio']=day_nt['order_number'].div(day_total['order_number'],axis=0)

In [ ]:
day_nt.drop('order_number',axis=1,inplace=True)

In [ ]:
day_nt.head()

,order_dow,ntre_ratio
0,0,0.414724
1,1,0.396157
2,2,0.410229
3,3,0.413728
4,4,0.409021


Merging dataframes of step 4.1 and 4.2

In [ ]:
user_day=user_day.merge(day_nt,on='order_dow',how='left')

In [ ]:
user_day.head()

,user_id,order_dow,reordered,ntre_ratio
0,1,1,0.882353,0.396157
1,1,2,0.444444,0.410229
2,1,3,0.545455,0.413728
3,1,4,0.727273,0.409021
4,2,1,0.671875,0.396157


In [ ]:
user_day.rename(columns={'reordered':'dow_re_ratio','ntre_ratio':'dow_ntre_ratio'},inplace=True)
user_day.head()

,user_id,order_dow,dow_re_ratio,dow_ntre_ratio
0,1,1,0.882353,0.396157
1,1,2,0.444444,0.410229
2,1,3,0.545455,0.413728
3,1,4,0.727273,0.409021
4,2,1,0.671875,0.396157


In [ ]:
user_day.to_csv('/content/drive/MyDrive/Instacart/uxd.csv')


5.   Product 'S Features Extraction



**5.1)Product's not reorder ratio**

Product not reorder ratio=
                        Total number of times the product not reorder
                        ------------------------------------------
                        Total number of times this product purchased by any user

In [ ]:
#selecting rows with reordered=0 i.e not reordered
ntre_df=combine_df[combine_df.reordered==0]
ntre_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,reverse_order_number
0,2539329,1,prior,1,2,8,0.0,196,1,0,10
1,2539329,1,prior,1,2,8,0.0,14084,2,0,10
2,2539329,1,prior,1,2,8,0.0,12427,3,0,10
3,2539329,1,prior,1,2,8,0.0,26088,4,0,10
4,2539329,1,prior,1,2,8,0.0,26405,5,0,10


In [ ]:
#total number of times a product  not reorder:given by number of reordered col(as it contains only 0- not reorder) per product_id in ntre_df
prd_nt_reorder=ntre_df.groupby('product_id')['reordered'].count().reset_index(name='prd_nt_reorders')

In [ ]:
prd_nt_reorder.head()

,product_id,prd_nt_reorders
0,1,716
1,2,78
2,3,74
3,4,182
4,5,6


In [ ]:
#total times product ordered - given by count of reordered per product _id 
prd_purchases=combine_df.groupby('product_id')['reordered'].count().reset_index(name='Total purchases')

In [ ]:
prd_purchases.head()

,product_id,Total purchases
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [ ]:
#required ratio
prd_nt_reorder['prd_ntre_ratio']=prd_nt_reorder['prd_nt_reorders'].div(prd_purchases['Total purchases'],axis=0)

In [ ]:
prd_nt_reorder.head()

,product_id,prd_nt_reorders,prd_ntre_ratio
0,1,716,0.386609
1,2,78,0.866667
2,3,74,0.267148
3,4,182,0.553191
4,5,6,0.400000


In [ ]:
prd_nt_reorder.drop('prd_nt_reorders',axis=1,inplace=True)
prd_nt_reorder.head()  #final df

,product_id,prd_ntre_ratio
0,1,0.386609
1,2,0.866667
2,3,0.267148
3,4,0.553191
4,5,0.400000


In [ ]:
del prd_purchases , ntre_df

**5.2)Average of product add to cart in last 5 orders**

            =Average of positions of product in cart in its last 5 purchases by all user.

In [ ]:
#last 5 orders of each product
last_5_orders=combine_df.groupby('product_id')['order_id'].nlargest(5).reset_index()
last_5_orders.head()

,product_id,level_1,order_id
0,1,9181333,3419644
1,1,27427027,3417477
2,1,24857437,3416714
3,1,13179246,3416680
4,1,5759239,3414875


In [ ]:
last_5_orders.shape

(244644, 3)

In [ ]:
#selecting rows with only last 5 orders per product from combine df 
prd_last=combine_df.merge(last_5_orders,on=['product_id','order_id'],how='inner')

In [ ]:
prd_last.shape

(244644, 12)

In [ ]:
del last_5_orders

In [ ]:
#avg of add to cart pos of last 5 orders of prd
prd_cart=prd_last.groupby('product_id')['add_to_cart_order'].mean().reset_index()

In [ ]:
prd_cart.head()

,product_id,add_to_cart_order
0,1,5.4
1,2,11.6
2,3,4.4
3,4,16.4
4,5,8.2


Merging dfs of step 5.1 amd 5.2

In [ ]:
prd_df=prd_cart.merge(prd_nt_reorder,on='product_id',how='left')

In [ ]:
prd_df.head()

,product_id,add_to_cart_order,prd_ntre_ratio
0,1,5.4,0.386609
1,2,11.6,0.866667
2,3,4.4,0.267148
3,4,16.4,0.553191
4,5,8.2,0.400000


In [ ]:
prd_df.rename({'add_to_cart_order':'prd_cart_last_5'},axis=1,inplace=True)

In [ ]:
prd_df.head()

,product_id,prd_cart_last_5,prd_ntre_ratio
0,1,5.4,0.386609
1,2,11.6,0.866667
2,3,4.4,0.267148
3,4,16.4,0.553191
4,5,8.2,0.400000


In [ ]:
prd_df.shape

(49677, 3)

In [ ]:
prd_df.to_csv('/content/drive/MyDrive/Instacart/prd.csv')